## CNN Architectures

이미지 분류에 있어서는 다양한 모형들이 ImageNet Challenge 를 통해서 나오게 되었습니다.

### AlexNet (2012)

Krizhevsky et al. 힌튼박사님의 박사과정 학생이였습니다.

- 2개의 평행한 Convolution flow를 설계하게 됩니다. 2개의 GPU를 사용했기 때문이라고 합니다.
- 224x224x3 의 input
- 첫 conv는 11x11사이즈의 96개의 filter를 적용했습니다. 
- 두번째 conv는 5x5
- 최초로 ReLU를 사용
- Norm layers : Local Response Normalization (LRN)
    - 활성화를 넘어서 2번째 레이어로 넘어간 부분에 대해서 Local적인 부분에 Normalization을 걸어주게 됩니다.
    - 오늘날에는 batch normalization을 하게됩니다.
- Heavy data augmentation
- Dropout : 0.5
- Batch Size : 128
- SGD Momentum : 0.9
- Learning rate with 1e-2 with anneling
- L2 weight decay : 5e-4
- 7CNN ensemble

### VGGNet (2014)

Simonyan and Zisserman

- Small filter(3x3) and deeper network
- Current standard deep CNN
- Stack of two 3x3 filter
    - Why? 3x3 filter를 두개 넣으면 5x5보다 파라미터가 작기 때문입니다.
- NO LRN
- ensembles
- Total Memory : 24M * 4 bytes ~= 96MB / image
- Total Paramas : 138M parameters

### GoogLeNet (2014)

Szegedy et al.

- Deeper network with computional efficiency
- No FC layer
- Inception module
    - Parallel filter multiple receptive field sizes
    - Reduce parameters using 1x1 conv?
        - 1x1 conv가 무엇일까요? 1x1 filter의 depth가 64개로 해서 depth를 줄이는 효과를 주게 됩니다.
    - Only 5M parameters : x12 less than VGGNet
    

### ResNet (2015)

- Very deep netwrok is hard to train!
    - 레이어를 쌓을 수록, test error가 떨어지지 않습니다. 
- Residual connections 과 Highway를 넣어줍시다.
    - Convolution 마다 Skip-connection을 뚫어주는 것이 아이디어 입니다.
    - 이때 Same convlution을 통해서 같은 차원을 유지해야합니다
    - 차원이 다를시에는 $a^{[l]}$에 linear transform을 해줄 wight를 곱해주어야합니다.
    - relu에 의해 항등함수를 학습하기 매우 쉽습니다, 수행능력이 떨어지지 않는다는 것을 보장할 수 있습니다.
- Loss function이 훨신 낮아보이게 됩니다.

$$
a^{[l+2]} = g(z^{[l+2]}+a^{[l]})
$$

### Network in network

1x1 convolution을 사용합니다. 일반적으로 Channel이 너무 클때, 이 채널의 지역적인 정보를 압축하고 싶을때 사용을 하게 됩니다. 또한 1x1를 쓰는것은 non-linearity를 주는 효과도 줄 수 있습니다. 

- 일반적으로 Channel이 1 이상인 경우 매우 중요한 역할을 합니다.
- 채널간의 지역적인 특성을 반영할 수 있다는 장점을 가지게 됩니다. 

### Inception network

1x1, 3x3, 5x5, pooling중에 어떤것이 좋은 컨볼루션일까요? Inception 모듈의 가장 큰 특징은 바로 이것입니다. 28x28x192짜리 input데이터를 다양한 Convolution과 pooling layer를 사용하여 28x28x256짜리 데이터로 변환시킵니다. 

하지만 이 계산은 높은 연산량을 시키게 됩니다. 만약 28x28x192를 5x5x32 Same Conv를 쓰게 된다면, 120M의 연산량이 발생하게 됩니다. 이러한 높은 연산량을 줄이기 위한 방법이 바로 위에서 소개한 1x1 Conv입니다.(bottleneck layer)

- 28x28x192
- Conv 1x1x192가 16개 존재하게 됩니다
- 28x28x16
- Conv 5x5x16이 32개를 배치합니다
- 28x28x32

120M에서 12.4M까지 연산량이 줄어들게 됩니다.

- Side branch를 사용하여 중간중간 output 예측을 하게 만들어 줍니다. 

### Visualization Weight

First Layer
- Simply visualizae each column of the matrix
    - Acoustic-level features : harmonic/percussive/how high energy
- Find songs that maximize the filter activations
    - The genres of the tracks in these playlists are quite different

Upper Layer
- Find songs that maximize the filter activation
- The Genres of the tracks become more similar in the playlists
- How can we visualizae the features in upper layer?

### Deconvolutional Network

데이터셋에서 Convolution을 거쳐서 feature Map을 하고 feedforward로 진행시킨다.

### DeconvNet

### Resnet Implementation

ResNet의 핵심모듈은 shortcut 혹은 skip connection입니다. 이는 2개의 큰 block으로 구성되게 됩니다. 첫번째는 identity blcok과 Convolution blcok입니다.

__The identity block__

Identity block은 Resnet의 기본적인 block이 됩니다. 일반적으로 우리가 구성하는 conv layer는 Convolution layer와 batch norm layer 그리고 ReLU로 구성됩니다. 하지만 Resnet의 Identity block은 input activation($a^{[l]}$)을 2레이어 뒤의 같은 차원의 output activation에 더해주는 것입니다. 

__Arguments__

X : 2차원의 이미지 데이터입니다. (Batch, n_H_prev, n_W_prev, n_C_prev)
f : integer, specifying the shape of the middle CONV's window for the main path
filters : python list of integers, defining the number of filters in the CONV layers of the main path
stage -- integer, used to name the layers, depending on their position in the network
block -- string/character, used to name the layers, depending on their position in the network

Returns:
X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
"""

In [1]:
import torch.nn as nn

In [4]:
def conv3x3(in_planes, out_planes, stride=1, groups=1,dilation=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=3, stride=stride, 
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=1, stride=stride,bias=False)

In [7]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1,downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups = 1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        
        # conv1 과 downsample layer는 샘플들의 정보를 점점 압축시킵니다.
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self,x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

In [8]:
class Bottleneck(nn.Module):
    expansion = 4
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(plane *(base_width/64.)) * groups
        # self.conv2 와 self.downsample layer는 downsample역할을 하게 된다.
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplanes=True)
        self.stride = stride
        
    def forward(self,x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out